
ВАЖНО! Чтобы ваше решение сохранялось и вы могли его нам предоставить в качестве ответа, сохраните себе копию ноутбука (Файл -> Сохранить копию на Диск).

Затем разрешите доступ к ноутбуку по ссылке (Поделиться -> Все у кого есть ссылка)

Вводная информация:

Мы занимаемся автоматизацией управления личных кабинетов клиентов, торгующих на маркетплейсах. В заданиях представлено 5 таблиц, отражающих упрощённые варианты источников, с которыми мы работаем ежедневно для расчёта параметров, составления отчетов, дашбордов и тд. В рамках заданий Вам необходимо проанализировать эти данные и представить результаты по каждой задаче.

С помощью данной тетрадки вы сможете ответить на вопросы тестового задания,используя для это SQL запросы или Python, в зависимости от личных предпочтений.  

В случае, если Вы не владеете навыками работы с SQL или Python, задания можно выполнять используя эксель. В этом случае мы настаиваем на выполнении дополнительных задач.

Для работы Вам будет полезно ознакомиться со структурой базы данных и описанием источников

[Структура БД](https://drive.google.com/file/d/1LRUrIpcgB4Bcv1kpSp-Vv7Gu3r4wTHTy/view?usp=drive_link)

[Описание БД](https://docs.google.com/document/d/1ms9ln6aZDRhM884Ddqr4ZUPQ_sryZ7VMWlX3EpEsuOg/edit?usp=sharing)

# SQL

In [493]:
import gdown
import pandas as pd
import subprocess
import os
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize' : (12, 5)}, font='verdana')
plt.style.use('dark_background')
import sqlite3 as sql3

Подгружаем базу

In [495]:
url='https://drive.google.com/file/d/1hAtTBGUnAQjp3yMIx-i1Qx3QJ3XFjZmv/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
gdown.download(url, 'test_bd_for_sql.db', quiet=False)
connection = sql3.connect('test_bd_for_sql.db')

Downloading...
From: https://drive.google.com/uc?id=1hAtTBGUnAQjp3yMIx-i1Qx3QJ3XFjZmv
To: c:\Users\mail\OneDrive\Documents\my_python\2_ad_hoc\testing_me\test_bd_for_sql.db
100%|██████████| 6.03M/6.03M [00:00<00:00, 20.5MB/s]


Смотрим на таблицы в базе

In [496]:
sql_query = """
            SELECT *
            FROM sqlite_master
            WHERE type = 'table'
            """
cursor = connection.cursor()
cursor.execute(sql_query)
a = cursor.fetchall()
t = []
for i in range(12):
    try:
        t.append(a[i][4].split("\"")[1])
    except IndexError:
        continue
t

['Nomenclature',
 'ComissionBySubjectName',
 'OrdersByDate',
 'WarehouseList',
 'StorageCostByDay']

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Продажами считаем заказы, которые не были отменены и имеют статус "Клиентский"

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную переменную, с которой вы будете работать в дальнейшем.

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

Изучим данные. Посмотрим на Nomenclature

In [497]:
query = """        SELECT * FROM Nomenclature LIMIT 2        """
pd.read_sql_query(query, connection)

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice
0,9428193,0,229720501,YORK,Щетки для стекол,342.92
1,9347354,0,98675218,YORK,Щетки для стекол,260.56


In [498]:
query = """ SELECT COUNT(NmId), COUNT(DISTINCT NmId) FROM Nomenclature   """
pd.read_sql_query(query, connection)

,COUNT(NmId),COUNT(DISTINCT NmId)
0,108,47


NmId не уникален и выполняет фунцию сопоставления в комбинации с SizeName

In [499]:
query = """ SELECT DISTINCT NmId, SizeName FROM Nomenclature ORDER BY NmId """
pd.read_sql_query(query, connection)

,NmId,SizeName
0,1362534,0
1,1407438,0
2,1472848,0
3,1668975,0
4,1927403,0
...,...,...
103,9174701,42
104,9174701,38
105,9326663,0
106,9347354,0


Все комбинации NmId и SizeName уникальны.  Итого 108 позиций в продажах

Посмотрим на таблицу OrdersByDate

In [ ]:
query = """ SELECT * FROM  OrdersByDate WHERE SizeName IS NULL """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,3747525,2024-11-16T12:46:11,344.0,0,Клиентский,210284,None
1,3757368,2024-11-16T12:47:47,352.0,0,Клиентский,210284,None
2,2496844,2024-11-16T12:51:41,396.0,0,Клиентский,210284,None
3,5767819,2024-11-16T12:58:37,797.0,0,Клиентский,206348,None
4,5767819,2024-11-16T12:58:37,797.0,0,Клиентский,206348,None
...,...,...,...,...,...,...,...
2010,2496844,2024-11-30T23:47:25,327.0,0,Клиентский,206348,None
2011,7059036,2024-11-30T23:49:11,874.0,0,Клиентский,206348,None
2012,7059036,2024-11-30T23:49:11,874.0,0,Клиентский,206348,None
2013,2496844,2024-11-30T23:59:58,327.0,0,Клиентский,206348,None


5675 записей, из которых у 2017 стоит NULL вместо размера

In [516]:
query = """ 
        SELECT * 
        FROM  Nomenclature as n
        LEFT JOIN 
         (  
        SELECT * 
        FROM  OrdersByDate
        WHERE SizeName IS NULL
        GROUP BY  NmId,SizeName    ) as t  
        ON n.NmId = t.NmId

        WHERE t.SizeName IS NULL AND t.date IS NOT NULL
        """
pd.read_sql_query(query, connection)

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,9428193,0,229720501,YORK,Щетки для стекол,342.92,9428193,2024-11-16T14:31:12,897.00,0,Клиентский,210284,None
1,9347354,0,98675218,YORK,Щетки для стекол,260.56,9347354,2024-11-16T20:24:09,758.00,0,Клиентский,120762,None
2,9326663,0,109595410,YORK,Щетки косметические,90.76,9326663,2024-11-16T15:55:39,301.00,0,Клиентский,206348,None
3,7156668,S,350075652,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
4,7156668,M,350075653,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
5,7156668,L,350075654,YORK,Перчатки хозяйственные,75.84,7156668,2024-11-18T22:23:30,344.00,0,Клиентский,117986,None
6,7059036,0,318908541,YORK,Салфетки для уборки,440.80,7059036,2024-11-22T03:15:17,873.00,0,Клиентский,206348,None
7,6032372,0,238666916,YORK,Швабры,820.48,6032372,2024-11-17T13:22:08,2386.00,0,Клиентский,210284,None
8,5767819,0,101177417,YORK,Щетки для уборки,252.65,5767819,2024-11-16T12:58:37,797.00,0,Клиентский,206348,None
9,5717244,0,98674253,YORK,Губки для посуды,49.56,5717244,2024-11-16T14:27:37,373.00,0,Клиентский,206348,None


Видим, что в таблице заказов не заполнена часть  размеров. При этом большая часть NULL - это не указанный 0, кроме артикула ", но мы уже не узнаем какой их размеров был выдан в заказе, поэтому при сборке джойнов все NULL в размерах заменим на размер 0

In [553]:
query = """ SELECT SUM(SoldPrice) FROM  OrdersByDate WHERE isCancel = 0 AND orderType = "Клиентский" """
pd.read_sql_query(query, connection)

,SUM(SoldPrice)
0,4323858.51


Напродавали в сумме на 4,3М

Посмотрим на таблицу WarehouseList

In [518]:
query = """  SELECT * FROM WarehouseList LIMIT 2 """
pd.read_sql_query(query, connection)

,Origid,Warehouse,WorkTime,NearCityName
0,218987,Алматы Атакент,24/7,Махачкала
1,204939,Астана,24/7,Петропавловск-Камчатский


In [519]:
query = """  SELECT COUNT(Origid), COUNT(DISTINCT Origid) FROM WarehouseList LIMIT 2 """
pd.read_sql_query(query, connection)

,COUNT(Origid),COUNT(DISTINCT Origid)
0,205,121


In [521]:
query = """  SELECT * FROM WarehouseList ORDER BY Origid DESC"""
pd.read_sql_query(query, connection)

,Origid,Warehouse,WorkTime,NearCityName
0,336442,Щербинка,24/7,Москва
1,333530,СЦ Симферополь Молодежненское,24/7,Симферополь
2,324108,Астана 2,24/7,Москва
3,321932,Чашниково,24/7,Москва
4,318294,СЦ Софьино,24/7,Москва
...,...,...,...,...
200,1193,Хабаровск,24/7,Хабаровск
201,686,Новосибирск,24/7,Новосибирск
202,686,Новосибирск,24/7,Новосибирск
203,507,Коледино,24/7,Москва


Часть записей в таблице задвоена, а значит при джойне надо будет убрать дубли

In [541]:
query = """  SELECT DISTINCT Origid, Warehouse FROM WarehouseList ORDER BY Origid DESC"""
pd.read_sql_query(query, connection)

,Origid,Warehouse
0,336442,Щербинка
1,333530,СЦ Симферополь Молодежненское
2,324108,Астана 2
3,321932,Чашниково
4,318294,СЦ Софьино
...,...,...
116,2737,Санкт-Петербург (Уткина Заводь)
117,1733,Екатеринбург - Испытателей 14г
118,1193,Хабаровск
119,686,Новосибирск


Кстати, о складах - в таблице заказов есть отгрузки cо склада с id = 0, тогда как в таблице складов такого склада нет

In [554]:
query = """ SELECT * FROM OrdersByDate WHERE warehouseID = 0 AND isCancel = 0 AND orderType="Клиентский" """
pd.read_sql_query(query, connection)

,NmId,date,SoldPrice,isCancel,orderType,warehouseID,SizeName
0,5134688,2024-11-02 21:22:14,636.0,0,Клиентский,0,0
1,5534114,2024-11-07 16:10:13,3890.0,0,Клиентский,0,0
2,5134688,2024-11-07 22:33:04,905.0,0,Клиентский,0,0
3,5134688,2024-11-08 00:59:33,905.0,0,Клиентский,0,0
4,3475731,2024-11-08 20:00:19,711.0,0,Клиентский,0,32
...,...,...,...,...,...,...,...
273,9326663,2024-11-30T14:13:07,301.0,0,Клиентский,0,None
274,9326663,2024-11-30T17:49:41,301.0,0,Клиентский,0,None
275,5717244,2024-11-30T21:26:42,258.0,0,Клиентский,0,None
276,9347354,2024-11-30T22:07:05,636.0,0,Клиентский,0,None


На 143413 отгрузили неизвестно откуда... На 4,3М как-будто немного, и все же

In [551]:
query = """ SELECT SUM(SoldPrice) FROM OrdersByDate WHERE warehouseID = 0 AND isCancel = 0 AND orderType="Клиентский" """
pd.read_sql_query(query, connection)

,SUM(SoldPrice)
0,143413.66


Посмотрим на таблицу StorageCostByDay

In [525]:
query = """  SELECT * FROM StorageCostByDay WHERE OfficeId < 0 ORDER BY OfficeId """
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
1,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,0.245385
2,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
3,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,0.245385
4,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,213459505,0.245385
...,...,...,...,...,...,...
947,2024-11-30,-1000033,Пальто,NELIY VINCERE,129045785,2.410800
948,2024-11-30,-1000033,Пальто,NELIY VINCERE,129067756,2.410800
949,2024-11-30,-1000033,Куртки,NELIY VINCERE,241060323,2.410800
950,2024-11-30,-1000033,Куртки,Riches,254332716,2.410800


Склад с отрицательным id...

In [526]:
query = """  SELECT * FROM StorageCostByDay WHERE WarehousePrice < 0 ORDER BY OfficeId """
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
1,2024-11-01,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
2,2024-11-01,-1000033,Майки спортивные,Flavour knit Gymnastic,213459505,-0.024539
3,2024-11-02,-1000033,Майки спортивные,Flavour knit Gymnastic,63309500,-0.024539
4,2024-11-02,-1000033,Шорты,Flavour knit Gymnastic,213459483,-0.024539
...,...,...,...,...,...,...
16254,2024-11-30,324108,Футболки,FRANK VANYA,126151327,-0.005075
16255,2024-11-30,324108,Майки спортивные,Flavour knit Gymnastic,213459548,-0.008678
16256,2024-11-30,324108,Купальники гимнастические,Flavour knit Gymnastic,213459615,-0.008678
16257,2024-11-30,324108,Купальники гимнастические,Flavour knit Gymnastic,319883878,-0.017357


... и отрицательные стоимости хранения...

In [528]:
query = """     SELECT * FROM StorageCostByDay WHERE ChrtId = "31789515" ORDER BY OfficeId limit 2"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice
0,2024-11-01,507,Водолазки,Flavour knit,31789515,0.1015
1,2024-11-01,507,Водолазки,Flavour knit,31789515,0.5075


... и полные дубли записей с разной стоимостью хранения

In [ ]:
query = """     SELECT *, MIN(WarehousePrice), MAX(WarehousePrice), MAX(WarehousePrice) - MIN(WarehousePrice) as diff
                FROM StorageCostByDay
                GROUP BY ChrtId, Date, OfficeId, brand, Subject
                HAVING diff <> 0             
"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice,MIN(WarehousePrice),MAX(WarehousePrice),diff
0,2024-11-01,507,Водолазки,Flavour knit,31789515,0.507500,0.101500,0.507500,0.406000
1,2024-11-01,120762,Водолазки,Flavour knit,31789515,0.563500,0.161000,0.563500,0.402500
2,2024-11-01,206348,Водолазки,Flavour knit,31789515,0.154000,0.077000,0.154000,0.077000
3,2024-11-02,507,Водолазки,Flavour knit,31789515,0.507500,0.101500,0.507500,0.406000
4,2024-11-02,120762,Водолазки,Flavour knit,31789515,0.563500,0.161000,0.563500,0.402500
...,...,...,...,...,...,...,...,...,...
15034,2024-11-12,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711
15035,2024-11-13,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711
15036,2024-11-14,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711
15037,2024-11-15,1733,Швабры,YORK,416723700,1.247344,-0.062367,1.247344,1.309711


In [ ]:
query = """     SELECT *, MIN(WarehousePrice), MAX(WarehousePrice), MAX(WarehousePrice) - MIN(WarehousePrice) as diff
                FROM StorageCostByDay
                GROUP BY Date, ChrtId, OfficeId, brand, Subject
                HAVING diff <> 0   
                Order by diff DESC   
                LIMIT 20
                   
"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehousePrice,MIN(WarehousePrice),MAX(WarehousePrice),diff
0,2024-11-22,206348,Щетки для уборки,YORK,98675540,234.57280,-11.728640,234.57280,246.301440
1,2024-11-23,206348,Щетки для уборки,YORK,98675540,229.74336,-11.487168,229.74336,241.230528
2,2024-11-24,206348,Щетки для уборки,YORK,98675540,220.08448,-11.004224,220.08448,231.088704
3,2024-11-25,206348,Щетки для уборки,YORK,98675540,206.97600,-10.348800,206.97600,217.324800
4,2024-11-26,206348,Щетки для уборки,YORK,98675540,200.76672,-10.038336,200.76672,210.805056
5,2024-11-27,206348,Щетки для уборки,YORK,98675540,193.17760,-9.658880,193.17760,202.836480
6,2024-11-28,206348,Щетки для уборки,YORK,98675540,185.58848,-9.279424,185.58848,194.867904
7,2024-11-29,206348,Щетки для уборки,YORK,98675540,179.37920,-8.968960,179.37920,188.348160
8,2024-11-01,120762,Щетки для уборки,YORK,98675540,171.10016,-17.110016,171.10016,188.210176
9,2024-11-02,120762,Щетки для уборки,YORK,98675540,160.75136,-16.075136,160.75136,176.826496


Итого имеем вот такие разбросы стоимости хранения для 15039 комбинаций даты, склада, товара, группы и размера. 

Причины такого положения вещей непонятны, поэтому принимаю управленческое решение и в качестве стоимости хранения выбираю максимальную стоимость. Если по факту откажется меньше - хозяин товара только обрадуется

In [535]:
query = """     SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                FROM StorageCostByDay
                GROUP BY ChrtId, Date, OfficeId, brand, Subject"""
pd.read_sql_query(query, connection)

,Date,OfficeId,Subject,Brand,ChrtId,WarehouseCost
0,2024-11-19,117501,Костюмы,LAVIRA,24078191,2.975000
1,2024-11-20,117501,Костюмы,LAVIRA,24078191,2.975000
2,2024-11-06,2737,Платья,LAVIRA,25556765,4.704000
3,2024-11-07,2737,Платья,LAVIRA,25556765,4.704000
4,2024-11-08,2737,Платья,LAVIRA,25556765,4.704000
...,...,...,...,...,...,...
51404,2024-11-12,1733,Швабры,YORK,416723700,1.247344
51405,2024-11-13,1733,Швабры,YORK,416723700,1.247344
51406,2024-11-14,1733,Швабры,YORK,416723700,1.247344
51407,2024-11-15,1733,Швабры,YORK,416723700,1.247344


С учетом вышесказанного соберем все вместе, не забыв исключить из записей возвраты и не "клиентские" заказы

In [ ]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.iscancel, ord.orderType, ord.warehouseID,
                            --nom.BrandName, nom.SubjectName, nom.CostPrice, 
                            nom.ChrtId,
                            com.Commission,  com.CategoryName,
                            war.Warehouse, war.Origid1 as Origid,
                            date(st.Date) as Date, st.ChrtId,  st.WarehouseCost
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
            WHERE             iscancel = 0 AND orderType = "Клиентский"
      
                              )
        
        SELECT              *
        FROM                cte
        WHERE              WarehouseCost IS NULL
      
        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,iscancel,orderType,warehouseID,ChrtId,Commission,CategoryName,Warehouse,Origid,Date:1,ChrtId:1,WarehouseCost
0,2024-11-04,4558489,46-48,903.0,0,Клиентский,210284,120692596.0,24.5,Одежда,Белая Дача,210284.0,None,None,None
1,2024-11-05,4558489,54-56,1308.0,0,Клиентский,210284,120692598.0,24.5,Одежда,Белая Дача,210284.0,None,None,None
2,2024-11-06,4558489,54-56,1003.0,0,Клиентский,210284,120692598.0,24.5,Одежда,Белая Дача,210284.0,None,None,None
3,2024-11-07,4558489,42-44,928.0,0,Клиентский,210284,123737052.0,24.5,Одежда,Белая Дача,210284.0,None,None,None
4,2024-11-08,4558489,42-44,1003.0,0,Клиентский,210284,123737052.0,24.5,Одежда,Белая Дача,210284.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2226,2024-11-30,9428193,0,870.0,0,Клиентский,210284,229720501.0,19.5,Хозяйственные товары,Белая Дача,210284.0,None,None,None
2227,2024-11-30,5717244,0,258.0,0,Клиентский,0,98674253.0,19.5,Хозяйственные товары,None,NaN,None,None,None
2228,2024-11-30,9347354,0,636.0,0,Клиентский,0,98675218.0,19.5,Хозяйственные товары,None,NaN,None,None,None
2229,2024-11-30,9428193,0,870.0,0,Клиентский,210284,229720501.0,19.5,Хозяйственные товары,Белая Дача,210284.0,None,None,None


Поскольку для многих дат нет соответствующих записей в таблице - половина костов имеют значение NULL

In [598]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, ord.warehouseID,
                            --nom.BrandName, nom.SubjectName, nom.CostPrice, 
                            nom.ChrtId,
                            com.Commission,  com.CategoryName,
                            st.WarehouseCost
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
            WHERE             iscancel = 0 AND orderType = "Клиентский"
      
                              )
        
        SELECT              *
        FROM                cte
        WHERE              WarehouseCost IS NULL
      
        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,warehouseID,ChrtId,Commission,CategoryName,WarehouseCost
0,2024-11-04,4558489,46-48,903.0,210284,120692596.0,24.5,Одежда,None
1,2024-11-05,4558489,54-56,1308.0,210284,120692598.0,24.5,Одежда,None
2,2024-11-06,4558489,54-56,1003.0,210284,120692598.0,24.5,Одежда,None
3,2024-11-07,4558489,42-44,928.0,210284,123737052.0,24.5,Одежда,None
4,2024-11-08,4558489,42-44,1003.0,210284,123737052.0,24.5,Одежда,None
...,...,...,...,...,...,...,...,...,...
2226,2024-11-30,9428193,0,870.0,210284,229720501.0,19.5,Хозяйственные товары,None
2227,2024-11-30,5717244,0,258.0,0,98674253.0,19.5,Хозяйственные товары,None
2228,2024-11-30,9347354,0,636.0,0,98675218.0,19.5,Хозяйственные товары,None
2229,2024-11-30,9428193,0,870.0,210284,229720501.0,19.5,Хозяйственные товары,None


Стоимость хранения определяется в первую очередь весом и габаритами, то есть - категорией товара. Выберем консервативную стратегию защиты прибыли и соберем не средние, а максимальные косты для категорий и проставим их вместо NULL в сводной таблице

In [610]:
query = """     SELECT Subject, MAX(WarehousePrice) as max
                FROM StorageCostByDay
                GROUP BY Subject
                ORDER BY max DESC   
                   
"""
pd.read_sql_query(query, connection)

,Subject,max
0,Щетки для уборки,234.572800
1,Стулья,195.336960
2,Карты географические,126.772800
3,Салфетки для уборки,92.133888
4,Полотенца банные,79.296000
...,...,...
71,Кремы,0.101500
72,Джемперы,0.101500
73,Декор для маникюра,0.101500
74,Браслеты,0.101500


Добавим недостающую стоимость хранения и подготовим итоговую таблицу

In [655]:
query = """
        WITH cte AS
            (SELECT         date(ord.date) as date, ord.NmId, ord.SizeName, ord.SoldPrice, nom.CostPrice, com.Commission,
                            IIF(WarehouseCost IS NULL, max.max, WarehouseCost)  as Cost,
                            com.CategoryName, max.Subject,nom.BrandName
            FROM            (SELECT NmId, date, SoldPrice, isCancel, orderType, warehouseID, COALESCE(SizeName, 0) as SizeName
                             FROM OrdersByDate
                             )
                             as ord
                                LEFT JOIN Nomenclature as nom 
                                  ON nom.NmId = ord.NmId AND ord.SizeName = nom.SizeName
                                LEFT JOIN ComissionBySubjectName as com 
                                  ON nom.SubjectName = com.SubjectName
                                LEFT JOIN (SELECT DISTINCT Origid as Origid1, Warehouse FROM WarehouseList) as war
                                  ON ord.warehouseID = war.Origid1
                                LEFT JOIN 
                                  (SELECT Date, OfficeId, Subject, Brand, ChrtId, MAX(WarehousePrice) as WarehouseCost
                                  FROM StorageCostByDay
                                  GROUP BY ChrtId, Date, OfficeId, brand, Subject
                                  ) as st
                                  ON  st.ChrtId = nom.ChrtId AND st.Officeid = war.Origid1 AND strftime('%d', ord.date) = strftime('%d', st.Date)
                                LEFT JOIN
                                  (SELECT Subject, MAX(WarehousePrice) as max
                                   FROM StorageCostByDay
                                   GROUP BY Subject) as max
                                  ON  max.Subject = nom.SubjectName
            WHERE             iscancel = 0 AND orderType = "Клиентский"
      
                              )
        
        SELECT              *
        FROM                cte
        ORDER BY date, NmId, SizeName
      
        """
pd.read_sql_query(query, connection)

,date,NmId,SizeName,SoldPrice,CostPrice,Commission,Cost,CategoryName,Subject,BrandName
0,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
1,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
2,2024-11-01,1362534,0,513.12,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
3,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
4,2024-11-01,1362534,0,529.00,181.52,22.5,15.7696,Красота,Щетки косметические,YORK
...,...,...,...,...,...,...,...,...,...,...
5130,2024-11-30,9428193,0,870.00,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK
5131,2024-11-30,9428193,0,870.05,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK
5132,2024-11-30,9428193,0,870.05,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK
5133,2024-11-30,9428193,0,870.00,342.92,19.5,28.7500,Хозяйственные товары,Щетки для стекол,YORK


### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа) за месяц, руб.
- рентабельность (маржинальность), %
- медианную прибыль (маржу) за месяц, руб.
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]

Задание 3
### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]


### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]

<b>Вывод</b>
```
Ваш вывод
```



### Дополнительное задание 2
<b>Визуализация</b>
```
Сделать дашборд с основными позателями, которые по Вашему мнению могут понадобиться владельцу кабинета для ежедневной работы
```
Примечание: Выполнение этого задания не обязательно в тетради. Вы можете скачать любую получившуюся у Вас таблицу и выполнить его в любом другом формате любым удобным для Вас способом (PowerBi, xls и др.)
Получившийся файл прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

In [ ]:
%%sql
-- Ваше решение

 * sqlite:////content/Test_BD_for_SQL.db
Done.


[]

```
Ссылка на дашборд
```

# Python

## Python - Загрузка датасетов

In [ ]:
import pandas as pd
import requests
import os
import gdown
from google.colab import files

In [ ]:
!gdown 'https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG'

Downloading...
From: https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG
To: /content/Test_BD_for_Python.xlsx
100% 2.31M/2.31M [00:00<00:00, 72.7MB/s]


In [ ]:
try:
    full_data = pd.ExcelFile('/content/Test_BD_for_Python.xlsx')
    sheet_names = full_data.sheet_names
    print(sheet_names)
    Nomenclature = full_data.parse(sheet_names[0])
    ComissionBySubject = full_data.parse(sheet_names[1])
    OrdersStatistic = full_data.parse(sheet_names[2])
    WarhouseName = full_data.parse(sheet_names[3])
    StorageAmount = full_data.parse(sheet_names[4])
    print("Листы успешно загружены")

except FileNotFoundError:
   print(f"Файл не найден в директории колаба. Пожалуйста, убедитесь, что предыдущая ячейка отработала.")

except Exception as e:
    print(f"Произошла ошибка при загрузке файла: {e}")

['Nomenclature', 'ComissionBySubjectName', 'OrdersByDate', 'WarehouseList', 'StorageCostByDay']
Листы успешно загружены


## Инструкция

После активации блока "Загрузка и подключение к базе данных" база данных была загружена в окружени colab:
~~~
Nomenclature - Справочник номенклатуры
WarehouseList - Справочник складов
OrdersByDate - Статистика по заказам
ComissionBySubjectName - Размер комиссии по категориям
StorageCostByDay - Данные по стоимости хранения
~~~

Сам файл BD появится в меню "Файлы" слева от рабочей области ноутбука. (Test_BD_for_Python.bd)

Отобразить первые/последние n строк можно через функцию display() и head()/tail()

In [ ]:
display(Nomenclature.tail(5))

,NmId,SizeName,ChrtId,BrandName,SubjectName,CostPrice
103,1927403,0,249897385,Azur,Фольга пищевая,29.16
104,1668975,0,235799189,YORK,Щетки кухонные,284.73
105,1472848,0,98674398,YORK,Швабры,1478.18
106,1407438,0,235786280,YORK,Щетки для стекол,394.60
107,1362534,0,258161521,YORK,Щетки косметические,181.52


Для объединения таблиц используйте необходимый тип объединения

```
# outer join
merged_outer = pd.merge(df1, df2, on='nmid', how='outer')
print("Outer join:\n", merged_outer)

# left join
merged_left = pd.merge(df1, df2, on='nmid', how='left')
print("\nLeft join:\n", merged_left)

# right join
merged_right = pd.merge(df1, df2, on='nmid', how='right')
print("\nRight join:\n", merged_right)
```



Сохранить запрос через переменную в эксель файл
~~~
df = pd.DataFrame(Nomenclature)
df.to_excel('/content/Nomenclature.xlsx', sheet_name='Nomenclature', index=False)
~~~

Или напрямую, после реализации сохранения в эксель, дополнив код вызовом загрузчика
~~~
#Скачать сохранённый отчёт из директории
files.download('/content/Nomenclature.xlsx')
~~~

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную переменную, с которой вы будете работать в дальнейшем.

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

In [1]:
#Ваш код

### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа), руб.
- рентабельность (маржинальность), %
- медианную цену заказа
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

In [2]:
#Ваш код

Задание 3
### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

In [3]:
#Ваш код


### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


In [4]:
#Ваш код

<b>Вывод</b>
```
Ваш вывод
```



### Дополнительное задание 2
<b>Визуализация</b>
```
Сделать дашборд с основными позателями, которые по Вашему мнению могут понадобиться владельцу кабинета для ежедневной работы
```
Примечание: Выполнение этого задания не обязательно в тетради. Вы можете скачать любую получившуюся у Вас таблицу и выполнить его в любом другом формате любым удобным для Вас способом (PowerBi, xls и др.)
Получившийся файл прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

In [5]:
#Ваш код

```
Ссылка на дашборд
```

# Excel

Прежде чем перейти к выполнению задания на локальном компьютере, необходимо нажать на кнопку "Выполнить"

<a href="https://imgbb.com/"><img src="https://i.ibb.co/nff863w/image.png" alt="image" border="0"></a>в левом верхнем углу ячейки, вы ее увидите если наведете мышь на соответствующую ячейку, либо воспользоваться комбинацией клавиш `Ctrl+Enter` или `Shift+Enter`

После окончания выполнения кода блока - Вам будет предложено скачать файл в формате xlsx.

Так же нужно ознакомиться с параграфом "Инструкция"  

## Excel - Загрузка файла

In [ ]:
!pip install pandas
!pip install requests

In [ ]:
import pandas as pd
import requests
import os
import gdown
from google.colab import files

In [ ]:
!gdown 'https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG'

Downloading...
From: https://drive.google.com/uc?id=1hRUnvJOdLGiZYDgGu3G73sLgkthkGsvG
To: /content/Test_BD_for_Python.xlsx
100% 2.31M/2.31M [00:00<00:00, 29.9MB/s]


In [ ]:
try:
    full_data = pd.ExcelFile('/content/Test_BD_for_Python.xlsx')
    sheet_names = full_data.sheet_names
    print(sheet_names)
    Nomenclature = full_data.parse(sheet_names[0])
    ComissionBySubject = full_data.parse(sheet_names[1])
    OrdersStatistic = full_data.parse(sheet_names[2])
    WarhouseName = full_data.parse(sheet_names[3])
    StorageAmount = full_data.parse(sheet_names[4])
    print("Листы успешно загружены")

except FileNotFoundError:
   print(f"Файл не найден в директории колаба. Пожалуйста, убедитесь, что предыдущая ячейка отработала.")

except Exception as e:
    print(f"Произошла ошибка при загрузке файла: {e}")

['Nomenclature', 'ComissionBySubjectName', 'OrdersByDate', 'WarehouseList', 'StorageCostByDay']
Листы успешно загружены


In [ ]:
# Скачать сохранённый отчёт из директории
files.download('/content/Test_BD_for_Python.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Инструкция

## Задания

В рамках задач мы принимаем, что прибыль (маржа) это цена продажи минус все расходы. Что считать расходами в данном примере, мы оставляем на Ваше усмотрение.

Статистика источников собрана за ноябрь 2024 г.

В рамках заданий Вам необходимо будет рассчитывать метрики по всей артикульной матрице. Рекомендуется сначала совместить все таблицы и сохранить результат в отдельную таблицу, с которой вы будете работать в дальнейшем.
Получившийся файл по каждой задаче прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

### Задание 1
<b>Рассчитать показатели:</b>
```
- среднюю цену заказа каждого артикула
- скорость заказов
```
В ответе вывести топ 5 артикулов по скорости заказов

Ссылка на файл с решением
```
Ваша ссылка
```

### Задание 2
<b>Рассчитать показатели на основе продаж:</b>
```
- прибыль (маржа), руб.
- рентабельность (маржинальность), %
- медианную цену заказа
- оборачиваемость (дн)
```
В ответе вывести топ 5 артикулов по прибыли в руб

Ссылка на файл с решением
```
Ваша ссылка
```

### Задание 3
<b>Рассчитать показатели:</b>
```
- Стоимость хранения за месяц
- Средняя стоимость хранения в день
```
В ответе вывести топ 5 артикулов по средней стоимости хранения

Ссылка на файл с решением
```
Ваша ссылка
```


### Дополнительное задание 1
<b>Анализ данных</b>
```
Проведите анализ получившейся статистики по кабинету.
Какие решения можно принять по работе с ассортиментом?
Учесть, что поставок не будет ближайшие 30 дней, а отсутствие остатка влечет к снижению рейтинга карточки товара.
Какие выводы можно сделать по брендам на онове всей статистики за ноябрь?
```


<b>Вывод</b>
```
Ваш вывод
```



### Дополнительное задание 2
<b>Визуализация</b>
```
Сделать дашборд с основными позателями, которые по Вашему мнению могут понадобиться владельцу кабинета для ежедневной работы
```
Примечание: Выполнение этого задания не обязательно в тетради. Вы можете скачать любую получившуюся у Вас таблицу и выполнить его в любом другом формате любым удобным для Вас способом (PowerBi, xls и др.)
Получившийся файл прикрепите к тетради посредством ссылки, не забыв предоставить к нему доступ, если это необходимо.

```
Ссылка на дашборд
```